In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# replace only year to get info for desirable year
response = requests.get("https://www.hockey-reference.com/leagues/NHL_2017_games.html")

In [3]:
soup = BeautifulSoup(response.content, features= "html.parser")

In [4]:
boxScores = []

for e in soup.find_all("tr"):
    try:
        boxScores.append(e.find("a").get("href"))
    except AttributeError:
        pass

In [5]:
len(boxScores)

1317

In [6]:
def refinedInfo(lst,split_el):
    name = ['Day&Time', 'Arena', 'Attendance', 'Game Duration']
    dct = dict(zip(name,[0]*len(name)))
    for n in lst:
        t = n.split(split_el) 
        
        if 'Attendance' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif 'Arena' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif 'Game' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif ('PM' in n) or ('AM' in n):
            dct['Day&Time'] = n
    
    
            
    return list(dct.values())

In [7]:
col_name = ['Team_1','Team_1_score','Team_2','Team_2_score','Day&Time', 'Arena', 'Attendance', 'Game Duration']
df = pd.DataFrame(columns= col_name)
for i,half_url in enumerate(boxScores):  
    url = 'https://www.hockey-reference.com' + half_url
    response_1 = requests.get(url)
    soup_1 = BeautifulSoup(response_1.content, features= "html.parser")
    var_imd = soup_1.title.get_text().split(',')[0]
    score = [s.get_text().replace('\n','') for s in soup_1.find_all('div',attrs= {'class':"scores"})]
    if ' at' in var_imd:
        teams = var_imd.split(' at ')[:2]
        txt_1 = soup_1.find_all("div", attrs= {'class':"scorebox_meta"})
        imd = [j.get_text() for j in txt_1[0].find_all("div")]
        imd_2 = refinedInfo(imd[:-1],': ')
        df.loc[i] = [teams[0]] + [score[0]] + [teams[1]] + [score[1]] +imd_2
    elif ' vs. ' in var_imd:
        teams = var_imd.split(' vs. ')[:2]
        txt_1 = soup_1.find_all("div", attrs= {'class':"scorebox_meta"})
        imd = [j.get_text() for j in txt_1[0].find_all("div")]
        imd_2 = refinedInfo(imd[:-1],': ')
        df.loc[i] = [teams[0]] + [score[0]] + [teams[1]] + [score[1]] +imd_2

In [8]:
df.shape

(1317, 8)

In [9]:
df

,Team_1,Team_1_score,Team_2,Team_2_score,Day&Time,Arena,Attendance,Game Duration
0,St. Louis Blues,5,Chicago Blackhawks Box Score — October 12,2,"October 12, 2016, 8:00 PM",United Center,"21,729",2:29
1,Calgary Flames,4,Edmonton Oilers Box Score — October 12,7,"October 12, 2016, 10:00 PM",Rogers Place,"18,347",2:35
2,Toronto Maple Leafs,4,Ottawa Senators Box Score — October 12,5,"October 12, 2016, 7:00 PM",Canadian Tire Centre,"17,618",2:36
3,Los Angeles Kings,1,San Jose Sharks Box Score — October 12,2,"October 12, 2016, 10:30 PM",SAP Center at San Jose,"17,562",2:29
4,Montreal Canadiens,4,Buffalo Sabres Box Score — October 13,1,"October 13, 2016, 7:00 PM",KeyBank Center,"19,070",2:44
5,Boston Bruins,6,Columbus Blue Jackets Box Score — October 13,3,"October 13, 2016, 7:00 PM",Nationwide Arena,"18,144",2:29
6,Anaheim Ducks,2,Dallas Stars Box Score — October 13,4,"October 13, 2016, 8:30 PM",American Airlines Center,"18,532",2:30
7,New Jersey Devils,1,Florida Panthers Box Score — October 13,2,"October 13, 2016, 7:30 PM",BB&T Center,"17,128",2:36
8,New York Islanders,3,New York Rangers Box Score — October 13,5,"October 13, 2016, 7:00 PM",Madison Square Garden (IV),"18,200",2:29
9,Washington Capitals,2,Pittsburgh Penguins Box Score — October 13,3,"October 13, 2016, 8:00 PM",PPG Paints Arena,"18,630",2:50


In [10]:
df['Team_2'] = df['Team_2'].apply(lambda x: x.split(' Box ')[0])

In [11]:
df

,Team_1,Team_1_score,Team_2,Team_2_score,Day&Time,Arena,Attendance,Game Duration
0,St. Louis Blues,5,Chicago Blackhawks,2,"October 12, 2016, 8:00 PM",United Center,"21,729",2:29
1,Calgary Flames,4,Edmonton Oilers,7,"October 12, 2016, 10:00 PM",Rogers Place,"18,347",2:35
2,Toronto Maple Leafs,4,Ottawa Senators,5,"October 12, 2016, 7:00 PM",Canadian Tire Centre,"17,618",2:36
3,Los Angeles Kings,1,San Jose Sharks,2,"October 12, 2016, 10:30 PM",SAP Center at San Jose,"17,562",2:29
4,Montreal Canadiens,4,Buffalo Sabres,1,"October 13, 2016, 7:00 PM",KeyBank Center,"19,070",2:44
5,Boston Bruins,6,Columbus Blue Jackets,3,"October 13, 2016, 7:00 PM",Nationwide Arena,"18,144",2:29
6,Anaheim Ducks,2,Dallas Stars,4,"October 13, 2016, 8:30 PM",American Airlines Center,"18,532",2:30
7,New Jersey Devils,1,Florida Panthers,2,"October 13, 2016, 7:30 PM",BB&T Center,"17,128",2:36
8,New York Islanders,3,New York Rangers,5,"October 13, 2016, 7:00 PM",Madison Square Garden (IV),"18,200",2:29
9,Washington Capitals,2,Pittsburgh Penguins,3,"October 13, 2016, 8:00 PM",PPG Paints Arena,"18,630",2:50
